# Snowflake Cortex AI と Managed MCP サーバーを使用したエージェント駆動ワークフローの構築

このノートブックでは、MCPツールを使用してLangGraphエージェントを構築する方法と、AIオブザーバビリティを使用してツールの説明を評価・改善する方法を紹介します。

## 概要

MCPツールを使用して以下をクエリするヘルスリサーチエージェントを構築します：
- 医学文献のためのPubMed
- 臨床試験情報のための臨床試験データベース

TruLensは自動的にすべてのツール呼び出しをトレースし、以下を表示します：
- 呼び出されているMCPツール
- 入力引数と出力
- 実行時間とエラー
- ダッシュボードでの完全な会話フロー

## セットアップ

まず、アカウント設定を更新して、すべてを許可するネットワークポリシーを作成し、`claude-sonnet-4-5`を呼び出すためのクロスリージョン推論を有効にします。これは、[`alter_account_settings.sql`](./alter_account_settings.sql)をコピーしてSnowflake SQLワークシートで実行することで行えます。

## 実行環境にPythonパッケージをインストール

以下のライブラリをインストールします：

- trulens-connectors-snowflake
- trulens-core
- trulens-providers-cortex
- trulens-apps-langgraph
- langchain-openai または langchain-snowflake
- langchain-mcp-adapters

🚨🚨 LLMの呼び出しに`langchain-snowflake`を使用する場合は、Python 3.9以上3.12未満のバージョンを使用する必要があります。Python環境がサポートされているバージョンを使用していることを確認してください。そうでないと、`langchain-snowflake`をインストールできません。🚨🚨

In [ ]:
!uv add trulens-connectors-snowflake trulens-core trulens-providers-cortex trulens-apps-langgraph 'langchain-openai==0.3.30' langchain-mcp-adapters 'langchain-snowflake==0.2.2'

## AI-ready データの取得

このステップでは、[Cortex Knowledge Extensions](https://docs.snowflake.com/en/user-guide/snowflake-cortex/cortex-knowledge-extensions/cke-overview)を通じてClinicalTrials.govとPubmed CentralからAI対応データにアクセスします。これはSnowflakeマーケットプレイスで利用可能な事前構築済みのCortex Search Servicesです。

1. [Clinical Trials Research Database](https://app.snowflake.com/marketplace/listing/GZSTZ67BY9ORD/snowflake-clinical-trials-research-database)のマーケットプレイスページを開きます

2. 「Get」を選択してます

3. リスティングから作成されたデータベースへのアクセスを許可するために、パブリックロールを追加します

4. 「Get」を選択します

5. 表示される確認ダイアログで、「Open」を選択して新しいタブでサンプルクエリを含むSnowsightワークシートを開くか、「Done」を選択します

6. [PubMed Biomedical Research Corpus](https://app.snowflake.com/marketplace/listing/GZSTZ67BY9OQW/snowflake-pubmed-biomedical-research-corpus?)のリスティングでステップ1-5を繰り返します

## SnowflakeでMCPサーバーを作成

[`create_mcp_server.sql`](./create_mcp_server.sql)をSnowflakeワークシートにコピーして実行します。これにより、前のステップで作成した2つのCortex Knowledge Extensionsを持つMCPサーバーが作成されます。

<div style="background-color:#ffe6e6; padding:20px; border-width:4px; border-color:#e07070; border-style:double; border-radius:8px; color:#a00000; font-weight:bold;">

## 🔑 APIキー設定：OpenAI（オプション）
    
🚨 **重要な注意事項:** **Snowflake Cortex LLMのみ**でエージェントを実行したい場合は、`OPENAI_API_KEY`をコメントアウトしたままにできます。このノートブックの後半でSnowflake Cortexモデルオプションを選択します。
    
</div>

## Snowflake（およびオプションでOpenAI）の認証情報を以下に追加してください。

Snowflakeの認証情報を取得するには、以下の手順に従ってください：

1. [こちら](https://app.snowflake.com/_deeplink/settings/authentication)からプログラマティックアクセストークン（PAT）を取得します
2. 左下のユーザープロフィールをクリックします
3. 「Connect a tool to Snowflake」を選択します
4. アカウント識別子の値をコピーし、SNOWFLAKE_ACCOUNTと、SNOWFLAKE_MCP_SERVER_URLの<account-id>の両方のプレースホルダーに貼り付けます
5. ログイン名の値をコピーし、SNOWFLAKE_USERのプレースホルダーに貼り付けます

OpenAIを使用する場合は、[こちら](https://platform.openai.com/api-keys)でAPIキーを作成または取得できます。

In [ ]:
import os
import logging

logging.getLogger().setLevel(logging.CRITICAL)

# Configure API keys and MCP server connection
# os.environ["OPENAI_API_KEY"] = "sk-proj-..."
os.environ["SNOWFLAKE_PAT"] = "..."
os.environ["SNOWFLAKE_ACCOUNT"] = "..."
os.environ["SNOWFLAKE_USER"] = "tsho"
os.environ["SNOWFLAKE_MCP_SERVER_URL"] = "https://<account-id>.snowflakecomputing.com/api/v2/databases/HEALTH_DB/schemas/PUBLIC/mcp-servers/HEALTH_MCP_SERVER"

In [ ]:
from snowflake.snowpark import Session
import os

snowflake_connection_parameters = {
    "account": os.getenv("SNOWFLAKE_ACCOUNT"),
    "user": os.getenv("SNOWFLAKE_USER"),
    "password": os.getenv("SNOWFLAKE_PAT"),
    "warehouse": "MCP_WH",
    "database": "HEALTH_DB",
    "schema": "PUBLIC",
}

snowpark_session = Session.builder.configs(
    snowflake_connection_parameters
).create()

## MCPクライアントの作成とツールの取得

`langchain_mcp_adapters`の`MultiServerMCPClient`を使用して、ヘルスリサーチMCPサーバーに接続し、利用可能なツールを取得します。


In [ ]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.graph import START
from langgraph.graph import MessagesState
from langgraph.graph import StateGraph
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition

client = MultiServerMCPClient({
    "health_research": {
        "transport": "streamable_http",
        "url": os.environ["SNOWFLAKE_MCP_SERVER_URL"],
        "headers": {"Authorization": f"Bearer {os.environ['SNOWFLAKE_PAT']}",
        },
    }
})
tools = await client.get_tools()

In [ ]:
for t in tools:
    print(f"- {t.name}: {t.description}") in tools

<div style="background-color:#e6f7ff; padding:15px; border-width:3px; border-color:#91d5ff; border-style:solid; border-radius:6px; **color:#003a61;**">
    
### 🟢 **オプション1：LangChain 経由のOpenAI**
    
**操作:** **OpenAI APIキー**（`gpt-4o`）を使用してLangChainを使用する場合は、以下のコードセルの**コメントを解除して実行**してください。
    
</div>

In [ ]:
# from langchain_openai import ChatOpenAI

# # Initialize the LLM
# model = ChatOpenAI(model="gpt-4o")

<div style="background-color:#f0f8ff; padding:15px; border-width:3px; border-color:#adc6ff; border-style:solid; border-radius:6px;">

### 🔵 **オプション2：LangChain経由のSnowflake Cortex**

**操作:** **Snowflake Cortex**（`claude-4-sonnet`）を使用してLangChainを使用する場合は、以下のコードセルの**コメントを解除して実行**してください。

</div>

In [ ]:
from langchain_snowflake import ChatSnowflake

# Initialize chat model
model = ChatSnowflake(
    session=snowpark_session, 
    model="claude-sonnet-4-5", 
    temperature=0.1, 
    max_tokens=1000
)

## LangGraphエージェントの構築

以下を持つLangGraphアプリケーションを作成します：
1. **call_model**ノード - どのツールを使用するかを決定するLLM
2. **tools**ノード - 選択されたMCPツールを実行
3. **tools_condition** - モデルとツール間のルーティング

グラフは、エージェントが質問に答えるのに十分な情報を得るまで、モデルとツールの間をループします。


In [ ]:
import asyncio

# Define the call_model function
async def call_model(state: MessagesState):
    response = await model.bind_tools(tools).ainvoke(state["messages"])
    return {"messages": response}


# Create the StateGraph
builder = StateGraph(MessagesState)
builder.add_node(call_model)
builder.add_node(ToolNode(tools))
builder.add_edge(START, "call_model")
builder.add_conditional_edges(
    "call_model",
    tools_condition,
)
builder.add_edge("tools", "call_model")
graph = builder.compile()

class Agent:
    def __init__(self, graph):
        import nest_asyncio
        nest_asyncio.apply()
        self.graph = graph

    async def ainvoke(self, messages):
        """Async version"""
        return await self.graph.ainvoke({"messages": messages})
    
    def invoke(self, messages):
        """Sync wrapper around async method"""
        return asyncio.run(self.ainvoke(messages))

agent = Agent(graph)

In [ ]:
from IPython.display import Image
#Image(agent.graph.get_graph().draw_mermaid_png(
#    max_retries=5, 
#    retry_delay=2.0
#))

print(agent.graph.get_graph().draw_ascii())

In [ ]:
agent.invoke("Xeljanzという薬の主な適応症は何ですか？")

In [ ]:
#response = agent.invoke("What is the primary indicator for the drug Xeljanz?")
response = agent.invoke("Xeljanzという薬の主な適応症は何ですか？")

print(response["messages"][-1].content)

## TruLensセッションの初期化

TruLensを設定して、トレースと評価をSnowflakeに保存します。

In [ ]:
from snowflake.snowpark import Session
from trulens.connectors.snowflake import SnowflakeConnector

sf_connector = SnowflakeConnector(snowpark_session=snowpark_session)

## ツール選択評価の作成

In [ ]:
from trulens.core.feedback.custom_metric import MetricConfig
from trulens.core.feedback.selector import Selector
from trulens.providers.cortex import Cortex

provider = Cortex(
    model_engine="claude-sonnet-4-5", snowpark_session=snowpark_session
)

f_tool_selection = MetricConfig(
    metric_name = "Tool Selection",
    metric_implementation = provider.tool_selection_with_cot_reasons,
    selectors={
        "trace": Selector(trace_level=True),
    },
)

f_tool_calling = MetricConfig(
    metric_name = "Tool Calling",
    metric_implementation = provider.tool_calling_with_cot_reasons,
    selectors={
        "trace": Selector(trace_level=True),
    },
)

metrics_to_compute = [
    f_tool_selection,
    f_tool_calling,
]

## TruLensでエージェント実行を記録

LangGraphアプリケーションを`TruGraph`でラップして、すべての実行を自動的に計装してトレースします。

TruLensは以下をキャプチャします：
- グラフ内の各ノード実行
- ツール名を含むMCPツール呼び出し（例：`pubmed_search`、`clinical_trials_search`）
- 各ステップでの入出力状態
- LLM生成呼び出し
- ツールルーティング決定

トレースは、エージェントの推論とツール使用の完全なフローを表示します。


In [ ]:
from trulens.apps.langgraph import TruGraph

tru_app = TruGraph(
    app=agent,
    app_name="healthcare_research_assistant",
    app_version="base",
    main_method=agent.invoke,
    connector=sf_connector,
)

In [ ]:
import pandas as pd

queries = ["公開された研究でセマグルチドとチルゼパチドはどのように比較されていますか？また、患者を募集中の直接比較臨床試験はありますか？",
"アルツハイマー病の最新の臨床試験は何ですか？",
"Xeljanzという薬の主な適応症は何ですか？"]

queries_df = pd.DataFrame(queries, columns=["query"])

## バッチ実行の設定

データセットの**バッチ実行**設定を行います。これにより以下が可能になります：
- 複数のエージェント呼び出しの実行
- 関連する評価の整理されたトラッキング
- 再現可能な実験のためのデータセット管理

In [ ]:
import uuid

from trulens.core.run import Run
from trulens.core.run import RunConfig

run_name = f"health_queries_run_{uuid.uuid4()}"

run_config = RunConfig(
    run_name=run_name,
    dataset_name="health_research_queries",
    source_type="DATAFRAME",
    dataset_spec={"RECORD_ROOT.INPUT": "query"},
)

run: Run = tru_app.add_run(run_config=run_config)


## バッチ実行の開始

バッチ実行を開始して、データセット内のすべてのクエリでエージェントを呼び出します。各呼び出しはトレースされ、Snowflakeに保存されます。

In [ ]:
run.start(input_df=queries_df)

## メトリクスの計算

In [ ]:
import time

while run.get_status() != "INVOCATION_COMPLETED":
    print(f"Status: {run.get_status()}")
    time.sleep(3)

print("Invocations complete, computing metrics...")
run.compute_metrics(metrics_to_compute)

In [ ]:
run.get_status()

---

# エージェントV2：改善されたツール説明

ベースエージェントには、TruLens評価を通じて特定された2つの重要な問題がありました：

## ベースバージョンの問題点

**1. Poor Tool Selection (Score: Low) - ツール選択の不備（スコア：低）**
- エージェントが事実に基づく医療クエリ（例：「Xeljanzの適応症は何ですか？」）でツールを使用しなかった
- 信頼できるソースの代わりに内部知識に依存
- 医療情報を検証する機会を逃した

**2. Tool Calling Errors (Multiple Failures) - ツール呼び出しエラー（複数の失敗）**
- エージェントが繰り返しカラム（`title`、`abstract`、`brief_description`、`disease_stage`）を指定しようとした
- これらのカラムはCortex Searchでインデックス化されていないため、エラーが発生
- エラー後も適応せず、同じ間違いを繰り返した
- エラー例：`"Column abstract was not indexed in this Cortex Search Service"`

## V2ソリューション：強化されたツール説明

V2 MCPサーバー（`health_mcp_server_v2`）は、改善されたツール説明でこれらの問題を修正します：

### 修正1：ツール使用の義務化
- ツールを**「信頼できる」**ソースとしてマーク
- 特定のクエリタイプに対する明示的な**「必須使用」**ステートメント
- 正確なユースケースをリスト：薬物適応症、有効性データ、安全性プロファイルなど

### 修正2：カラム指定エラーの防止
- 明確な指示：**「'query'パラメータのみを提供」**
- エージェントは利用できないカラムを検索しなくなる

### 期待される改善
- ✅ より高いツール選択スコア（エージェントが一貫してツールを使用）
- ✅ より高いツール呼び出しスコア（カラムエラーがなくなる）

## 必要なセットアップ

V2セルを実行する前に、改善されたMCPサーバーを作成してください：
```sql
-- Snowflakeでcreate_mcp_server_v2.sqlを実行
```


## エージェントV2の実装

改善されたツール説明を持つV2エージェントを構築してテストしましょう。

**前提条件：** Snowflakeで`create_mcp_server_v2.sql`を実行して、`health_mcp_server_v2` MCPサーバーを作成していることを確認してください。

## 改善されたMCPサーバーでV2クライアントを作成

In [ ]:
# Connect to the V2 MCP server
os.environ["SNOWFLAKE_MCP_SERVER_V2_URL"] = os.environ["SNOWFLAKE_MCP_SERVER_URL"].replace(
    "health_mcp_server", "health_mcp_server_v2"
)

client_v2 = MultiServerMCPClient({
    "health_research_v2": {
        "transport": "streamable_http",
        "url": os.environ["SNOWFLAKE_MCP_SERVER_V2_URL"],
        "headers": {"Authorization": f"Bearer {os.environ['SNOWFLAKE_PAT']}"},
    }
})
tools_v2 = await client_v2.get_tools()

## エージェントV2の構築

In [ ]:
# Build model with V2 tools
model_v2 = ChatSnowflake(
    session=snowpark_session, 
    model="claude-sonnet-4-5", 
    temperature=0.1, 
    max_tokens=1000
)

async def call_model_v2(state: MessagesState):
    response = await model_v2.bind_tools(tools_v2).ainvoke(state["messages"])
    return {"messages": response}

builder_v2 = StateGraph(MessagesState)
builder_v2.add_node(call_model_v2)
builder_v2.add_node(ToolNode(tools_v2))
builder_v2.add_edge(START, "call_model_v2")
builder_v2.add_conditional_edges("call_model_v2", tools_condition)
builder_v2.add_edge("tools", "call_model_v2")
graph_v2 = builder_v2.compile()

agent_v2 = Agent(graph_v2)

## V2エージェントをTruGraphでラップ

In [ ]:
tru_app_v2 = TruGraph(
    app=agent_v2,
    app_name="healthcare_research_assistant-build",
    app_version="v2_improved_descriptions",
    main_method=agent_v2.invoke,
    connector=sf_connector,
)

## 同じクエリでV2エージェントを実行

In [ ]:
run_name_v2 = f"health_queries_run_v2_{uuid.uuid4()}"

run_config_v2 = RunConfig(
    run_name=run_name_v2,
    dataset_name="health_research_queries",
    source_type="DATAFRAME",
    dataset_spec={"RECORD_ROOT.INPUT": "query"},
)

run_v2: Run = tru_app_v2.add_run(run_config=run_config_v2)

In [ ]:
run_v2.start(input_df=queries_df)

## V2のメトリクスを計算

In [ ]:
import time

In [ ]:
while run_v2.get_status() != "INVOCATION_COMPLETED":
    print(f"V2 Status: {run_v2.get_status()}")
    time.sleep(3)

print("V2 invocations complete, computing metrics...")
run_v2.compute_metrics(metrics_to_compute)

## 結果の比較

Health Research Agentの比較タブにアクセスすることで、MCPサーバーの両バージョン（ツール説明変更前と変更後）でエージェントがどのように動作するかを比較できます。

![agent_version_comparison.png](agent_version_comparison.png)

指定されたユースケースでのツール使用を義務化し、検索パラメータを指定するようにツール説明を更新することで、エージェントは最初のバージョンで発生していたツール選択とツール呼び出しの失敗を解決できるようになりました。